¡Absolutamente! Ahora que tenemos todos los requisitos claros y confirmados, aquí tienes la guía definitiva y detallada, paso a paso, para construir tu asistente de programación por WhatsApp con éxito.

Vamos a seguir el plan desde cero hasta tenerlo funcionando.

---

### **Guía Definitiva: Tu Asistente de Programación por WhatsApp (Costo Cero)**

Este tutorial te llevará a través de cada clic y cada línea de código necesarios.

#### **Fase 1: Preparación del Entorno en n8n (`localhost`)**

**Objetivo:** Instalar el conector necesario para WhatsApp en tu n8n.

1.  **Inicia n8n:** Asegúrate de que tu n8n esté corriendo en tu computadora.
2.  **Abre n8n en tu navegador:** Ve a `http://localhost:5678`.
3.  **Ve a "Community Nodes":**
    *   En el menú de la izquierda, haz clic en el ícono del engranaje (⚙️ **Settings**).
    *   En la nueva vista, selecciona **Community Nodes**.
4.  **Instala el Nodo de WhatsApp:**
    *   En la barra de búsqueda, escribe: `n8n-nodes-whatsapp-personal`.
    *   Aparecerá el nodo en la lista. Haz clic en el botón azul **Install**.
    *   Confirma la instalación en la ventana emergente. n8n te notificará que la instalación fue exitosa y que debes reiniciar la instancia.
5.  **Reinicia n8n:** Cierra la terminal donde estabas ejecutando n8n (con `Ctrl + C`) y vuelve a iniciarlo con el mismo comando que usas siempre. Este paso es **crucial** para que el nuevo nodo se cargue.

---

#### **Fase 2: Creación y Configuración del Flujo de Trabajo**

**Objetivo:** Construir el flujo nodo por nodo y conectar toda la lógica.

**Paso 1: Crear el Flujo y el Disparador (Trigger) de WhatsApp**

1.  **Crea un Flujo Nuevo:** Vuelve a la pantalla principal de n8n (`http://localhost:5678/workflows`) y haz clic en **Add workflow**.
2.  **Añade el Trigger:** Haz clic en el `+` grande y busca el nodo **`WhatsApp, personal Trigger`**. Selecciónalo.
3.  **Conecta tu WhatsApp:**
    *   En el panel derecho, en el campo `Credentials`, haz clic en **- Create New -**.
    *   Se abrirá una ventana. Dale un nombre a tu credencial, como `MiWhatsAppPersonal`, y haz clic en **Connect my account**.
    *   **¡Aparecerá un código QR!**
    *   **En tu teléfono:** Abre WhatsApp, ve a `Configuración > Dispositivos vinculados > Vincular un dispositivo` y escanea el código QR que se muestra en n8n.
    *   Espera unos segundos. n8n mostrará un mensaje de "Connection successful!". Haz clic en **Save** para guardar la credencial.
4.  **Configura el Evento:** En el campo `Events`, asegúrate de que esté seleccionado `Message (upsert)`. Esto activará el flujo con cada mensaje nuevo.

**Paso 2: Preparar el Prompt para la IA con el Nodo `Set`**

1.  **Añade el Nodo `Set`:** Haz clic en el `+` después del nodo de WhatsApp y busca y selecciona el nodo **`Set`**.
2.  **Configura el `Set`:**
    *   El nodo `Set` por defecto mantiene los datos que entran (`Keep Only Set` está desactivado), lo cual es perfecto.
    *   En la sección `Values to Set`, haz clic en **Add Value**.
    *   En el campo **Name**, escribe: `prompt_para_gemini`.
    *   En el campo **Value**, pega el siguiente texto. Este es el "cerebro" de tu asistente.

        ```
        Eres un asistente experto en programación para mis alumnos. Tu tono es amigable y didáctico. Responde a la siguiente pregunta de forma clara, concisa y siempre que sea posible, incluye un ejemplo de código simple. No respondas a preguntas que no estén relacionadas con programación, en ese caso di: "Lo siento, solo puedo ayudarte con dudas de programación".

        Pregunta del alumno: "{{ $json.body.message }}"
        ```
    *   La parte `{{ $json.body.message }}` se reemplazará automáticamente por el mensaje real del alumno.

**Paso 3: Conectar con Gemini para Generar la Respuesta**

1.  **Añade el Nodo `Google Gemini`:** Haz clic en el `+` después del nodo `Set` y busca y selecciona el nodo **`Google Gemini`**.
2.  **Conecta tu API Key de Gemini:**
    *   En el panel derecho, en `Credentials`, haz clic en **- Create New -**.
    *   Dale un nombre a la credencial, como `MiGeminiKey`.
    *   Pega tu **API Key de Gemini** en el campo `Google Gemini API Key`.
    *   Haz clic en **Save**.
3.  **Configura la Petición a Gemini:**
    *   Asegúrate de que `Resource` esté en **`Chat`**.
    *   En el campo `Prompt`, vamos a usar el valor que preparamos en el paso anterior. Haz clic en el engranaje ⚙️ junto al campo, selecciona **Add Expression**, y en el editor de expresiones, elige la variable del nodo `Set`:
        `{{ $('Set').item.json.prompt_para_gemini }}`

**Paso 4: Enviar la Respuesta de Vuelta por WhatsApp**

1.  **Añade el Nodo `WhatsApp, personal` (Sender):** Haz clic en el `+` después del nodo `Google Gemini` y busca de nuevo el nodo **`WhatsApp, personal`**.
2.  **Configura el Envío:**
    *   En `Credentials`, selecciona la credencial que ya creaste en el primer paso (`MiWhatsAppPersonal`).
    *   En `Operation`, asegúrate de que esté en **`Send Message`**.
    *   En `Chat ID`, tenemos que decirle a quién responder. Usa una expresión para obtener el número del remitente original. Haz clic en el engranaje ⚙️ y selecciona la ruta desde el **primer nodo** (el trigger):
        `{{ $('WhatsApp, personal Trigger').item.json.body.from }}`
    *   En `Message`, vamos a poner la respuesta de Gemini. Haz clic en el engranaje ⚙️ y selecciona la ruta correcta desde el nodo de **Gemini**:
        `{{ $('Google Gemini').item.json.candidates[0].content.parts[0].text }}`

---

#### **Fase 3: ¡Prueba y Activación!**

1.  **Guarda tu Flujo:** Dale un nombre memorable en la parte superior izquierda, como "Asistente de Programación WhatsApp".
2.  **Activa el Flujo:** Haz clic en el interruptor **`Inactive`** en la esquina superior derecha para cambiarlo a **`Active`**. ¡Tu flujo ahora está escuchando!
3.  **Realiza la Prueba Final:**
    *   Usa **otro número de WhatsApp** (el de un amigo, familiar o un segundo número tuyo).
    *   Envía un mensaje al número de WhatsApp que vinculaste a n8n.
    *   **Ejemplo de pregunta:** `¿Cuál es la diferencia entre let y const en JavaScript?`
    *   En unos segundos, ¡deberías recibir una respuesta generada por Gemini directamente en tu chat!

¡Felicidades! Si has seguido todos estos pasos, ahora tienes un asistente de IA personal y completamente funcional, construido por ti mismo y sin gastar un centavo.

---

**¿Qué sigue? ¡Tú decides!**

1.  **"¡Funcionó! ¿Cómo puedo añadir el mensaje 'Estoy pensando...' para mejorar la experiencia?"**
2.  **"Me gustaría guardar un registro de todas las preguntas y respuestas. ¿Podemos usar Google Sheets para eso?"**
3.  **"¿Qué pasa si alguien envía una imagen o un audio? ¿Cómo puedo manejar eso?"**